# IBA Probe for ECMP Traffic Imbalance

In this tutorial we will demonstrate the "ECMP Traffic Imbalance" probe that is used to messure traffic load on your ECMP interfaces and detect imbalances between links.

The first thing we need to do to is setup our AOS server and identify the blueprint we want to use.

In [1]:
aos_server = '172.20.91.3'
blueprint_name = "rack-based-blueprint-f03bfadb"

Next let's create a AOS python client that we will use to create our probe

In [2]:
import json
from aospy.swagger import Client

aos = Client('https://' + aos_server)

Now let's use that client to get the blueprint, so we can have the blueprint_id value we will need when we post the probe.

In [3]:
from aospy.swagger.groups.blueprints import Blueprints
bp_index = Blueprints(aos)
bp = bp_index[blueprint_name]

In [4]:
print bp.id

rack-based-blueprint-f03bfadb


Now let's import the ECMP imbalance probe from the library

In [7]:
from aospy.ibaprobelib.ecmp_imbalance import ecmp_imbalance_probe

Next let's create the ECMP imbalance probe payload information that we will need.  The probe takes a few parmateres which we can see if we introspect the function:

In [ ]:
?ecmp_imbalance_probe

The following paramaters are required:

`label` - Descriptive name for the probe  
`average_period` - period over which to average input Bps counter samples  
`duration` - number of seconds of recent-history in which we will consider ECMP imbalance  
`threshold_duration` - sum total of number of seconds of recent-history for which set of ECMP links must be unbalanced for anomaly to be raised  
`std_max` - maxiumum standard deviation in Bps across a set of ECMP paths on a given system      If this std deviation is exceeded, we consider that system to be imbalanced  
`anomaly_history_count` - number of anomaly flaps that will be recorded for inspection  
`max_systems_imbalanced` - if this number of total imbalanced systems is exceeded, an anomaly   is raised  
`system_imbalance_history_count` - the number of samples of recent-history for which we   maintain data about the number of systems that are imbalanced  


So for the case of reuse, we will define these variables here:

In [8]:
probe_label = 'DC1-ECMP-Imbalance'
average_period = 30
duration = 120
threshold_duration = 30
std_max = 12500000
anomaly_history_count = 500
max_systems_imbalanced = 2
system_imbalance_history_count = 5000

Now let's create the body data from these variables:

In [9]:
probe_body = ecmp_imbalance_probe(label=probe_label, average_period=average_period, duration=duration, threshold_duration=threshold_duration, std_max=std_max, anomaly_history_count=anomaly_history_count, max_systems_imbalanced=max_systems_imbalanced, system_imbalance_history_count=system_imbalance_history_count)

If you'd like to dump the contents of the probe_body, you can use the following:

In [ ]:
# print json.dumps(probe_body, indent=2)

Finally, post the probe body to AOS to create the probe.

In [10]:
probes_url = bp.url + "/probes"

In [11]:
probes_url

u'https://172.20.91.3/api/blueprints/rack-based-blueprint-f03bfadb/probes'

In [12]:
resp = aos.remote.api.post(probes_url, json=probe_body)

In [13]:
resp.ok

True

Now we have our IBA Probe ID:

In [14]:
probe_id = resp.json()['id']

print probe_id

1895d762-1619-43e0-9f6f-911de52228a0


And now we can see the IBA active probe in the UI:

<img src="media/ecmp_imbalance1.png">

And then select the probe on the UI to get the details:

<img src="media/ecmp_imbalance2.png">

# Delete the IBA ECMP Traffic Imbalance Probe

If you want to remove the IBA probe, you can use the API to do so, like the following; just uncomment this out.

In [15]:
resp = aos.remote.api.delete(probes_url + "/%s" % probe_id)
resp.ok

True